# Assignment 4 - Navigation

**Group assignment**

*15 points*

Group members:
* Name Surname - githubuser
* Name Surname - githubuser
* Name Surname - githubuser


In this assignment you will work with the turtlebot in the Gazebo simulation. Install Gazebo Ignition on your Ubuntu 24.04 ROS Jazzy system.

Use additional material on sensors here: [ROS Humble Binder with Gazebo Classic](https://binder.intel4coro.de/v2/gh/IntEL4CoRo/iis-exercises/dev?urlpath=lab%2Ftree%2F03_sensors%2Foffice.ipynb). It is not graded, but contains helpful material for working with sensors and sending commands. Start with the `office.ipynb` to create a Gazebo sim online. Then move on to the `sensors.ipynb` for understanding the laserscan before solving this Assignment.

## Local System Setup
(Courtesy by Kishan Ravindra Sawant)
- Create a new workspace (e.g., named `ros2_ws`) where you'll clone or develop all your ROS packages. Inside this workspace, create a folder named `src/`. Then, from within the `src/` directory, clone the following repositories,
```
git clone https://github.com/secorolab/create3_sim.git
git clone https://github.com/secorolab/turtlebot4.git
git clone https://github.com/secorolab/turtlebot4_simulator.git
```
- Install the following packages
```
sudo apt update 
sudo apt install -y ros-jazzy-control-msgs ros-jazzy-irobot-create-nodes ros-jazzy-turtlebot4-desktop ros-jazzy-controller-manager ros-jazzy-gz-ros2-control
```
- From the root of your workspace (i.e., ros2_ws/), build the workspace by executing `colcon build --allow-overriding $(colcon list --names-only)`
- **Note:** Generally `colcon build` is sufficient to build the packages, but as some of these are also included in the *underlay* workspace, they should be explicitly built with the flag *--allow-overriding*
- Every time you build the workspace, you need to **source** it so that your terminal session can recognize the linked libraries. To do this, execute `source install/setup.bash` from the root of your workspace

### Simulation
- Run TurtleBot4 in simulation
```
ros2 launch turtlebot4_gz_bringup turtlebot4_gz.launch.py world:=office_area
```
- Note: For first instance it will take time for the robot and the environment to load. Even after~15 minutes if you do not see the robot in the environment, then rerun the above launch script

- Check the laser scan. In the begining of the message you will see it has a 360 degree laser.
```
ros2 topic echo /scan --once
```


- To move the robot around, ether use the teleoperation interface from the simulator or execute the following command from the terminal.
```
ros2 run teleop_twist_keyboard teleop_twist_keyboard --ros-args -p stamped:=true
```


## Task 1 - Avoid collision

Let the robot move forwards. If there is a collision ahead turn on the spot (left or right) until it is safe to move forward again. Continue moving forward. 

Listen to the `/scan` topic to identify the collision. Connect that detection to a publisher to `/cmd_vel_unstamped`. See the tutorial referenced above for inspiration.

Save the behavior as Python file.

## Task 2 - Follow the wall

Write another behavior. This time follow the right wall at a fixed distance.

Save the behavior as Python file.

## Task 3 - Record a map

This task can be developed independently from Task 2 and 3. You need the workspace set up as described in the Local System Setup.

### Mapping
- We will use Simultaneous Localization and Mapping (SLAM) for robust mapping. Before executing SLAM, check if laser scanner data is being published by echoing to the /scan topic: `ros2 topic echo /scan --once`. If it is being published, then execute the SLAM node
```
ros2 launch turtlebot4_navigation slam.launch.py use_sim_time:=true
```
- To visualize the mapping process in rviz, execute the following command
```
ros2 launch turtlebot4_viz view_navigation.launch.py use_sim_time:=true
```
- Now move the robot around in the gazebo world to map the indoor environment completely
- Once the mapping is completed, from the path `turtlebot4_simulator/turtlebot4_gz_bringup/maps` execute the following command to save the map. Change <map_name> to desired name of the map
```
ros2 run nav2_map_server map_saver_cli -f <map_name> --occ 0.65 --free 0.15 --ros-args -p save_map_timeout:=20.0
```
There should now be 2 files, one .pgm and one .yaml in your current directory.

### Localization
- Localization can be achieved by two ways, one via SLAM when the map is partially known, and other Localization algorithms when a map already exists. We will look into Adaptive Monte-Carlo Localization (AMCL) algorithm for localization
- After saving the map, you can stop the mapping node. In a new terminal run the localization node
```
ros2 launch turtlebot4_navigation localization.launch.py map:=<complete_path_to_map_yaml_config_file>
```
- Now use the `2D Pose Estimate` from RViz and drag the arrow in the approximate location where the robot is placed. Move the robot around and observe the robot localizing in the environment

Submit the 2 map files.

## Solution

Submit the 2 Python files for Task 1 and Task 2 and the 2 files resulting from Task 3. 

To test your solution 
- follow the Local System Setup to have Gazebo
- run the localization with your map
- start the behavior from Task 1
- cancel Task 1 and start behavior of Task 2